In [1]:
import os
import pandas as pd
import matplotlib as plt
%matplotlib inline

# Cleaning Manually Typed In Data by WD Students
For years, work duty students have courageously attempted to type in data from peabody cards into a spreadsheet. Their labor is central to our OCR endeavor. Yet the data given to us by the Peabody is muddy. Work Duty students are generally exhausted and disinterested.

## Finding the CSV

In [2]:
# The root path of the directory
# i.e., the folder containing all files related to this project. 
root_path = os.getcwd()
os.listdir(root_path)

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'Card Catalog Processing.ipynb',
 'Moe.ipynb',
 'Otsu_Thresholding_Cropping.ipynb',
 'pdftotiff.sh',
 'peabody_files',
 'README.md',
 'Segmentations.ipynb',
 'test.png',
 'transcription.csv',
 'util.sh']

In [3]:
data_path = os.path.join(root_path, 'peabody_files')
os.listdir(data_path)

['.DS_Store', 'Accession Files', 'Peabody_Extended_Fields_Acc.1-12.csv']

`Accession Files` contain PDFs, but what is the csv?
## Examining the CSV and Identifying Relevant Columns
### Examining CSV

In [4]:
csv_name = os.listdir(data_path)[2]
ext_df = pd.read_csv(os.path.join(data_path, csv_name)) #Short for extended fields
pd.set_option("display.max_columns", len(ext_df.keys())) #Display all columns
ext_df.sample(10)

ACCESSNO  ACQVALUE  APPNOTES  APPRAISOR CAPTION CAT      CATBY  \
563          1       0.0       NaN        NaN     NaN   O  Work Duty   
1030         4       0.0       NaN        NaN     NaN   O  Work Duty   
600          2       0.0       NaN        NaN     NaN   O  Work Duty   
326          3       0.0       NaN        NaN     NaN   O  Work Duty   
569          1       0.0       NaN        NaN     NaN   O  Work Duty   
797          2       0.0       NaN        NaN     NaN   O  Work Duty   
773          2       0.0       NaN        NaN     NaN   O  Work Duty   
87           3       0.0       NaN        NaN     NaN   O  Work Duty   
548          1       0.0       NaN        NaN     NaN   O  Work Duty   
59           3       0.0       NaN        NaN     NaN   O  Work Duty   

         CATDATE      CATTYPE  CLASSES COLLDATE  COLLECTION COLLECTOR  \
563   02/23/2016  Archaeology      NaN      / /         NaN       NaN   
1030  05/16/2016  Archaeology      NaN      / /         NaN       NaN   
600   03/28/2016  Archaeology      NaN      / /         NaN       NaN   
326   05/03/2016  Archaeology      NaN      / /         NaN       NaN   
569   02/23/2016  Archaeology      NaN      / /         NaN       NaN   
797   04/05/2016  Archaeology      NaN      / /         NaN       NaN   
773   04/04/2016  Archaeology      NaN      / /         NaN       NaN   
87           / /  Archaeology      NaN      / /         NaN       NaN   
548   02/22/2016  Archaeology      NaN      / /         NaN       NaN   
59    04/20/2016  Archaeology      NaN      / /         NaN       NaN   

     CONDDATE CONDEXAM CONDITION CONDNOTES  CREATOR  CREATOR2  CREATOR3  \
563       / /      NaN       NaN       NaN      NaN       NaN       NaN   
1030      / /      NaN       NaN       NaN      NaN       NaN       NaN   
600       / /      NaN       NaN       NaN      NaN       NaN       NaN   
326       / /      NaN       NaN       NaN      NaN       NaN       NaN   
569       / /      NaN       NaN       NaN      NaN       NaN       NaN   
797       / /      NaN       NaN       NaN      NaN       NaN       NaN   
773       / /      NaN       NaN       NaN      NaN       NaN       NaN   
87        / /      NaN       NaN       NaN      NaN       NaN       NaN   
548       / /      NaN       NaN       NaN      NaN       NaN       NaN   
59        / /      NaN       NaN       NaN      NaN       NaN       NaN   

      CREDIT  CURVALMAX  CURVALUE  DATASET DATE  \
563      NaN        0.0       0.0      NaN  NaN   
1030     NaN        0.0       0.0      NaN  NaN   
600      NaN        0.0       0.0      NaN  NaN   
326      NaN        0.0       0.0      NaN  NaN   
569      NaN        0.0       0.0      NaN  NaN   
797      NaN        0.0       0.0      NaN  NaN   
773      NaN        0.0       0.0      NaN  NaN   
87       NaN        0.0       0.0      NaN  NaN   
548      NaN        0.0       0.0      NaN  NaN   
59       NaN        0.0       0.0      NaN  NaN   

                                                DESCRIP  DIMTYPE  DISPVALUE  \
563                                           Bone awl.        2        NaN   
1030                                 Quartzite scraper.        2        NaN   
600                                4 Quartz Arrowpoints        2        NaN   
326              Celt. Original card: Short stone adze.        2        NaN   
569                                           Bone awl.        2        NaN   
797                               1 large quartz knife.        2        NaN   
773                       Large broken shale spearpoint        2        NaN   
87    Basalt point with excurvate edges, side notche...        2        NaN   
548                                        Broken arrow        2        NaN   
59                                             Pottery.        2        NaN   

      EARLYDATE  EVENT  EXHIBITNO EXHLABEL1  EXHLABEL2 EXHLABEL3  EXHLABEL4  \
563           0    NaN          0       NaN        NaN       NaN        

### Preliminary Eliminations of Columns

This seems like an incomprehensible table, so I would benefit from identifying which columns contain valuable information. My first impression from the data is that **there are a lot of NaNs**. The most useful columns to me would not be dominated by NaNs. 

In [5]:
ext_df.isnull().sum().describe(percentiles=[x*0.1 for x in range(1, 9)])

count     249.000000
mean      711.638554
std       504.345869
min         0.000000
10%         0.000000
20%         0.000000
30.0%       4.000000
40%      1054.000000
50%      1093.000000
60.0%    1094.000000
70%      1094.000000
80%      1094.000000
max      1094.000000
dtype: float64

From this I understand that half of the **60%** of the **249 columns** have 1054 or more NaNs, which brings down the potential relevant columns down to 100 columns.

In [6]:
# Store 100 columns with the least NaNs
fewer_nan_cols = ext_df.isnull().sum().sort_values().index[:100]

My second impression from this table is that **some columns contain the same values for all rows.** This would automatically disqualify a column from containing relevant data.

In [7]:
# Store columns whose values vary
unique_cols = []
for col in ext_df.columns:
    unique_vals = ext_df[col].unique()
    unique_count = len(unique_vals)
    if unique_count !=1:
        unique_cols.append(col)

From these two impressions, I can whittle down the number of columns to review from 249 to 60. These columns are shown below:

In [8]:
cols_to_review = [col for col in fewer_nan_cols if col in unique_cols]
ext_df[cols_to_review].head(3)

ACCESSNO WEBINCLUDE      UPDATEDBY     UPDATED UDF18 UDF13  STATUSDATE  \
0         3        .F.  Taylor, Marla  05/10/2016   / /     0  05/10/2016   
1         3        .F.  Taylor, Marla  05/10/2016   / /     0  05/10/2016   
2         3        .F.  Taylor, Marla  05/10/2016   / /     0  05/10/2016   

        RECAS OBJECTID MAINTDATE  DEPTHFT  WIDTHIN  WIDTHFT  WIDTH  WEIGHTLB  \
0  Expedition   3/1000       / /      0.0      0.0      0.0    0.0       0.0   
1  Expedition   3/1001       / /      0.0      0.0      0.0    0.0       0.0   
2  Expedition   3/1002       / /      0.0      0.0      0.0    0.0       0.0   

   WEIGHTIN  WEIGHT REPATNOTIC  DEPTH  LENGTHIN  LENGTH EXCAVADATE  DEPTHIN  \
0       0.0       0        / /    0.0       0.0     0.0        / /      0.0   
1       0.0       0        / /    0.0       0.0     0.0        / /      0.0   
2       0.0       0        / /    0.0       0.0     0.0        / /      0.0   

   LENGTHFT  IMAGENO CONDDATE     CATDATE      CATTYPE EXHSTART COLLDATE  \
0       0.0        0      / /  05/10/2016  Archaeology      / /      / /   
1       0.0        0      / /  05/10/2016  Archaeology      / /      / /   
2       0.0        0      / /  05/10/2016  Archaeology      / /      / /   

                           DESCRIP          GPARENT          PARENT  \
0                      Pot sherds.  12: Archaeology         Ceramic   
1  Fragment of chipped arrowpoint.  12: Archaeology  Stone, Chipped   
2                      Pot sherds.  12: Archaeology         Ceramic   

             OBJNAME                                         PROVENANCE  \
0              Sherd  Section number - 6E Trench Number - IC.1 to IC...   
1  Point, Projectile  Provenience Description - From scrap Section n...   
2              Sherd  Section number - 7E Trench Number - IC.3 Taken...   

   SITENO             SITE MATERIAL   STATUSBY   STATUS      CATBY  \
0  42.001  Nevin Shellheap  Ceramic  Work Duty  Entered  Work Duty   
1  42.001  Nevin Shellheap    Stone  Work Duty  Entered  Work Duty   
2  42.001  Nevin Shellheap  Ceramic  Work Duty  Entered  Work Duty   

                                     ORIGIN OLDNO  \
0  United States | ME | Hancock | Blue Hill   A31   
1  United States | ME | Hancock | Blue Hill   A32   
2  United States | ME | Hancock | Blue Hill   A33   

                              NOTES  COUNT OTHERNO HOMELOC LOCFIELD1 STRATUM  \
0  July 2013 PastPerfect Conversion    0.0     NaN     NaN       NaN     NaN   
1  July 2013 PastPerfect Conversion    1.0     NaN    RSPM       NBS       8   
2  July 2013 PastPerfect Conversion    0.0     NaN     NaN       NaN     NaN   

  LEVEL UDF3 UNIT PEOPLE EXCAVATEBY  LOCFIELD2  LOCFIELD3 REPATTYPE PUBNOTES  \
0   NaN  NaN  NaN    NaN        NaN        NaN        NaN       NaN      NaN   
1   NaN  NaN  NaN    NaN        NaN       11.0       16.0       NaN      NaN   
2   NaN  NaN  NaN    NaN        NaN        NaN        NaN       NaN      NaN   

  UDF4 UDF21  
0  NaN   NaN  
1  NaN   NaN  
2  NaN   NaN

### Comparing CSV data with graphic data

With the table above, it is still difficult to identify which columns are helpful to building our OCR model and which are not. At this point, we can manually compare our cards and the dataset to see which columns contain data that correlates with the card text.

As Jupyter Notebooks are built on web standards, they allow the display of PDFs. For convenience I'll write quick functions:

In [9]:
from IPython.display import HTML
    
def card_path(acc_no, cat_no):
    return './peabody_files/Accession Files/%d_%04d.pdf' % (acc_no, cat_no)

def disp_card(acc_no, cat_no):
    return HTML('<iframe src=\'%s\' width=700 height=350></iframe>' % card_path(acc_no, cat_no)) 

def disp_cards(*args):
    '''
    Pass in multiple tuples. Tuples should be (acc_no, cat_no)
    '''
    html = ''
    for tup in args:
        frame = '<iframe src=\'%s\' width=700 height=350></iframe>' % card_path(*tup)
        html += frame
    return HTML(html)    

With the functions above, I can compare which columns have text that is also in the cards. Crucially, I notice that the OBJECTID column corresponds to 'Access number / category number', which helps me compare specific cards to rows.

In [10]:
ext_df[cols_to_review].sample(4)

ACCESSNO WEBINCLUDE      UPDATEDBY     UPDATED       UDF18 UDF13  \
482         3        .F.      Work duty  05/10/2016  07/30/1936     0   
449         3        .F.      Work duty  05/11/2016         / /     0   
340         3        .F.      Work duty  05/09/2016         / /     0   
2           3        .F.  Taylor, Marla  05/10/2016         / /     0   

     STATUSDATE       RECAS OBJECTID MAINTDATE  DEPTHFT  WIDTHIN  WIDTHFT  \
482  08/29/2015  Expedition    3/986       / /      0.0      0.0      0.0   
449  05/11/2016  Expedition    3/976       / /      0.0      0.0      0.0   
340  05/09/2016  Expedition    3/863       / /      0.0      0.0      0.0   
2    05/10/2016  Expedition   3/1002       / /      0.0      0.0      0.0   

     WIDTH  WEIGHTLB  WEIGHTIN  WEIGHT  REPATNOTIC  DEPTH  LENGTHIN  LENGTH  \
482    0.0       0.0       0.0       0  04/28/2015    0.0       0.0     0.0   
449    0.0       0.0       0.0       0         / /    0.0       0.0     0.0   
340    0.0       0.0       0.0       0         / /    0.0       0.0     0.0   
2      0.0       0.0       0.0       0         / /    0.0       0.0     0.0   

    EXCAVADATE  DEPTHIN  LENGTHFT  IMAGENO CONDDATE     CATDATE      CATTYPE  \
482        / /      0.0       0.0        0      / /         / /  Archaeology   
449        / /      0.0       0.0        0      / /  05/11/2016  Archaeology   
340        / /      0.0       0.0        0      / /  05/09/2016  Archaeology   
2          / /      0.0       0.0        0      / /  05/10/2016  Archaeology   

    EXHSTART COLLDATE                                            DESCRIP  \
482      / /      / /  Ochre-stained metapodial awl. Approx. 17cm lon...   
449      / /      / /                                         Potsherds.   
340      / /      / /                   Distal end of antler perforator.   
2        / /      / /                                        Pot sherds.   

             GPARENT          PARENT     OBJNAME  \
482  12: Archaeology  Bone, Modified  Perforator   
449  12: Archaeology         Ceramic       Sherd   
340  12: Archaeology  Bone, Modified  Perforator   
2    12: Archaeology         Ceramic       Sherd   

                                            PROVENANCE  SITENO  \
482  Provenience Description - Grave 2. Human remai...  42.001   
449  Section number - 9E Trench Number - IC.2 Site ...  42.001   
340  Section number - 8E Trench Number - IC.4 Excav...  42.001   
2    Section number - 7E Trench Number - IC.3 Taken...  42.001   

                SITE         MATERIAL       STATUSBY         STATUS  \
482  Nevin Shellheap  Faunal Material  Sousa, Bonnie  Deaccessioned   
449  Nevin Shellheap          Ceramic      Work Duty        Entered   
340  Nevin Shellheap  Faunal Material      Work Duty        Entered   
2    Nevin Shellheap          Ceramic      Work Duty        Entered   

         CATBY                                    ORIGIN OLDNO  \
482        NaN  United States | ME | Hancock | Blue Hill   474   
449  Work Duty  United States | ME | Hancock | Blue Hill   A23   
340  Work Duty  United States | ME | Hancock | Blue Hill   119   
2    Work Duty  United States | ME | Hancock | Blue Hill   A33   

                                                 NOTES  COUNT OTHERNO HOMELOC  \
482                   July 2013 PastPerfect Conversion    1.0     NaN    RSPM   
449                   July 2013 PastPerfect Conversion    0.0     NaN     NaN   
340  Douglas Byers (Principal Investigator, 00/00/3...    1.0     NaN    RSPM   
2                     July 2013 PastPerfect Conversion    0.0     NaN     NaN   

    LOCFIELD1 STRATUM                        LEVEL  \
482       NBS     NaN         Shell flecked humus.   
449       NaN       4                 Second peel.   
340       NBS      13  E side of hearth 3 in shell   
2         NaN     NaN                          NaN   

                             UDF3    UNIT          PEOPLE      EXCAVATEBY  \
482  Late Archaic Moorehead Phase   IC/7E 

In [11]:
disp_card(3, 1000)

Just for the card with Acc. No. 3 and Cat. No. 1000, I carefully determine that the columns ACCESSNO, OBJECTID, DESCRIP, PROVENANCE, SITENO, SITE, ORIGIN, OLDNO are relevant. I'll try this out for two more random cards

In [12]:
a = ext_df[cols_to_review].sample(2)
a

ACCESSNO WEBINCLUDE  UPDATEDBY     UPDATED UDF18 UDF13  STATUSDATE  \
672         2        .F.  Work duty  03/29/2016   / /     0  03/29/2016   
281         3        .F.  Work duty  05/03/2016   / /     0  05/03/2016   

          RECAS OBJECTID   MAINTDATE  DEPTHFT  WIDTHIN  WIDTHFT  WIDTH  \
672  Expedition    2/154  03/29/2016      0.0      0.0      0.0    0.0   
281  Expedition    3/776         / /      0.0      0.0      0.0    0.0   

     WEIGHTLB  WEIGHTIN  WEIGHT REPATNOTIC  DEPTH  LENGTHIN  LENGTH  \
672       0.0       0.0       0        / /    0.0       0.0     0.0   
281       0.0       0.0       0        / /    0.0       0.0     0.0   

    EXCAVADATE  DEPTHIN  LENGTHFT  IMAGENO CONDDATE     CATDATE      CATTYPE  \
672        / /      0.0       0.0        0      / /  03/29/2016  Archaeology   
281        / /      0.0       0.0        0      / /  05/03/2016  Archaeology   

    EXHSTART COLLDATE                                            DESCRIP  \
672      / /      / /                           Broken quartz arrowpoint   
281      / /      / /  Base of antler. Original cad: piece of cut ant...   

             GPARENT            PARENT            OBJNAME  \
672  12: Archaeology    Stone, Chipped  Point, Projectile   
281  12: Archaeology  Antler, Modified        Antler Tool   

                                            PROVENANCE  SITENO  \
672  East end of Squibnocket Pond, Martha's Vineyar...   M49/2   
281  Section number - 7E Trench Number - IC.1 Excav...  42.001   

                      SITE         MATERIAL   STATUSBY   STATUS      CATBY  \
672  Hornblower Shell Heap            Stone  Work Duty  Entered  Work Duty   
281        Nevin Shellheap  Faunal Material  Work Duty  Entered  Work Duty   

                                       ORIGIN OLDNO  \
672                                       NaN   NaN   
281  United States | ME | Hancock | Blue Hill   391   

                                                 NOTES  COUNT OTHERNO HOMELOC  \
672                                                NaN    NaN    S/17     NaN   
281  Douglas Byers (Principal Investigator, 00/00/3...    1.0     NaN    RSPM   

    LOCFIELD1 STRATUM                                    LEVEL  \
672       NaN     NaN                                      NaN   
281       NBS      10  In black humus under second shell layer   

                             UDF3            UNIT          PEOPLE  \
672                           NaN             NaN             NaN   
281  Late Archaic Moorehead Phase  IC1/7E (E end)  Byers, Douglas   

         EXCAVATEBY  LOCFIELD2  LOCFIELD3 REPATTYPE PUBNOTES UDF4 UDF21  
672             NaN        NaN        NaN       NaN      NaN  NaN   NaN  
281  Byers, Douglas        NaN        NaN       NaN      NaN  NaN   NaN

In [13]:
disp_cards((3, 911), (2, 283))

From the three above
- **ACCESSNO** has the Acc. No. verbatim
- **OBJECTID** has 'Acc. No/Cat. No.' verbatim
- **DESCRIP** matches Name (most of the time. Not during abbreviations, like 2/283)
- **OLDNO** and **OTHERNO** matches Orig.No. It's likely that if one is NaN, the other is filled.
- **PROVENANCE** matches card's 'Locality,' 'Situation,' and 'Remarks' but formatted weirdly.
- **SITENO** seems to correlate with cards' 'Site No.' category, but not sure how.
- **SITE** matches card's 'Site' category, but text is formatted differently. (Nevin Shellheap and Nevin Shell Heap)
- **ORIGIN** matches Locality when it's there. Weird format.
- **NOTES** matches remarks sometimes 

## Cleaning the data
Now that the relevant columns have been identified, it's time to clean the data. **The objective of this is to process the text in the csv to match the text on the cards as closely as possible.**

### Observations

In [14]:
cols_to_review = ['ACCESSNO','OBJECTID','DESCRIP','OLDNO','OTHERNO','PROVENANCE','SITENO','SITE','ORIGIN','NOTES']
ext_df[cols_to_review].head(10)

ACCESSNO OBJECTID                                            DESCRIP OLDNO  \
0         3   3/1000                                        Pot sherds.   A31   
1         3   3/1001                    Fragment of chipped arrowpoint.   A32   
2         3   3/1002                                        Pot sherds.   A33   
3         3   3/1003                            Beaver tooth implement.   A34   
4         3   3/1005  Barbed harpoon fragment. Appears to be unilate...   A37   
5         3   3/1006                     2 fragments stone arrow points    13   
6         3   3/1007                              1 chipped stone knife   13A   
7         3   3/1008                              Butt of adze? Surface   13b   
8         3   3/1009                       Edge of polished slate tool.   13C   
9         3   3/1010                                     Large plummet.     6   

  OTHERNO                                         PROVENANCE  SITENO  \
0     NaN  Section number - 6E Trench Number - IC.1 to IC...  42.001   
1     NaN  Provenience Description - From scrap Section n...  42.001   
2     NaN  Section number - 7E Trench Number - IC.3 Taken...  42.001   
3     NaN  Section number - 8E Trench Number - ID.1&2 Tak...  42.001   
4     NaN  Provenience Description Bluehill Falls, Me. - ...  42.001   
5     NaN                       Bluehill Falls, Me. Surface.  42.001   
6     NaN                       Bluehill Falls, Me. Surface.  42.001   
7     NaN                       Bluehill Falls, Me. Surface.  42.001   
8     NaN                       Bluehill Falls, Me. Surface.  42.001   
9     NaN             Bluehill Falls, Me. On beach, surface.  42.001   

              SITE                                    ORIGIN  \
0  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
1  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
2  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
3  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
4  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
5  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
6  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
7  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
8  Nevin Shellheap  United States | ME | Hancock | Blue Hill   
9  Nevin Shellheap  United States | ME | Hancock | Blue Hill   

                                               NOTES  
0                   July 2013 PastPerfect Conversion  
1                   July 2013 PastPerfect Conversion  
2                   July 2013 PastPerfect Conversion  
3                   July 2013 PastPerfect Conversion  
4  Not found in NBS 37/11. New location updated 1...  
5                   July 2013 PastPerfect Conversion  
6                   July 2013 PastPerfect Conversion  
7                   July 2013 PastPerfect Conversion  
8                   July 2013 PastPerfect Conversion  
9                   July 2013 PastPerfect Conversion

In [15]:
df = ext_df[cols_to_review]
df.isnull().sum()

ACCESSNO        0
OBJECTID        0
DESCRIP         1
OLDNO         541
OTHERNO       585
PROVENANCE      2
SITENO          7
SITE            8
ORIGIN        406
NOTES         562
dtype: int64

In [16]:
for col in df.columns:
    print(col, len(df[col].unique()), sep=' | ')

ACCESSNO | 12
OBJECTID | 1094
DESCRIP | 877
OLDNO | 518
OTHERNO | 363
PROVENANCE | 761
SITENO | 13
SITE | 14
ORIGIN | 8
NOTES | 38


Additional takeaways:
- `ACCESSNO` and `OBJECTID` are the most complete and reliable dataset.
- `DESCRIP` is a close second in completeness, but has so much variety that it would be difficult to eliminate differences from cards.
- `SITENO` and `SITE` have few NaNs and few variety, making them nearly as ideal as `ACCESSNO` and `OBJECTID`.
- That `OLDNO` and `OTHERNO` have high number of NaNs suggests that when one is NaN, the other is filled.

**I will start by focusing on Acc. No., Cat. No, and Site. **

### Acc. No. and Cat. No.
Goal: A new column `CATEGORYNO` holds result of text-processed `OBJECTID` showing only category number.

#### Checking Assumptions Before Text processing
My assumption is that all the values in `OBJECTID` follow the format ACCNO/CATNO. I want to check if this is true.

In [17]:
# Check if all values in OBJECTID column follow the format number/number
import re
follows_format = df.OBJECTID.apply(lambda s: bool(re.search('\d+\/\d+', s)))
follows_format.all()

False

Not all the values follow the format. What do the exceptions look like?

In [18]:
i = follows_format[follows_format == False].index
df.iloc[i]

ACCESSNO OBJECTID                                            DESCRIP  \
515          7   Jul-78                                                NaN   
520          1    1-Jan                                  Butt of arrowhead   
521          1    2-Jan                               Butt of quartz knife   
522          1    3-Jan                                 Crude quartz point   
523          1   14-Jan                       Point of large quartz blade.   
524          1    4-Jan                             Crude quartzite spear.   
525          1   15-Jan   Quartz arrow head, thick butt point chipped off.   
526          1    5-Jan                                     Crude chopper.   
527          1    6-Jan                  Crude rectangular quartz scraper.   
528          1   16-Jan        Red quartzite (?) arrowhead, straight butt.   
529          1    7-Jan                            Crude quartz arrowhead.   
530          1   17-Jan                    Crude arrow point, not stemmed.   
531          1    8-Jan                                Crude broken point.   
532          1   18-Jan                  Crude knife (?), quartz, stemmed.   
533          1    9-Jan                                     Rubbing stone.   
534          1   10-Jan                       Edge of crude chipped blade.   
535          1   11-Jan                      Rounded butt of quartz knife.   
536          1   19-Jan                                Quartz arrow point.   
537          1   12-Jan                          Point of crude arrowhead.   
538          1   20-Jan                         Butt of large crude knife.   
539          1   13-Jan                       Point of large quartz blade.   
540          1   21-Jan                           Broken quartz arrowhead.   
541          1   23-Jan                                   Crude arrowhead.   
542          1   Jan-35                                   One quartz arrow   
543          1   22-Jan                           Broken quartz arrowhead.   
544          1   24-Jan                                 Butt of arrowhead.   
545          1   Jan-36                                       Quartz arrow   
546          1   25-Jan                                       Quartz Knife   
547          1   26-Jan                                       Quartz Knife   
548          1   Jan-37                                       Broken arrow   
...        ...      ...                                                ...   
590          1   Jan-61                                 Piece of bone awl.   
591          1   Jan-62                                        Cut antler.   
592          2   Feb-85                        2 small quartz arrowpoints.   
593          1   Jan-63                           Scraps of crude pottery.   
594          1   Jan-64                 Two pieces of undecorated pottery.   
595          1   Jan-65                  First shell layer above scallops.   
596          1   Jan-66                             Small piece of pottery   
597          2   Feb-76                            Small quartz arrowpoint   
598          2   Feb-77                               2 Quartz Hammerstone   
600          2   Feb-78                               4 Quartz Arrowpoints   
601          2   Feb-79                        5 Fragments of quartz tools   
603          2   Feb-80                                     Quartz scraper   
604          2   Feb-81              1 arrowpoint and 1 butt of arrowpoint   
606          2   Feb-82                            Fragment of shale knife   
608          2   Feb-83                           Piece of worked hematite   
610          2   Feb-84                                  Quartz turtleback   
612          2   Feb-86                         2 fragments of crude tools   
614          2   Feb-87  Part of large broken knife 1 quartz arrowpoint...   
618          2   Feb-89                           Broken quartz arrowpoint   
619          2   Feb-90     

Hypothesis: If values in `OBJECTID` don't follow the Number/Number format, they follow the Month-Number or Number-Month format. 
Let's test this:

In [19]:
months_abbv = ['Jan','Feb','March','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
re_months = '(%s)' % ('|'.join(months_abbv))
t = df.iloc[i].OBJECTID.apply(lambda s: bool(re.match('(%s\-\d+)|(\d+\-%s)'%(re_months, re_months), s)))
t[t==False]

1013    False
Name: OBJECTID, dtype: bool

In [20]:
df.iloc[1013]

ACCESSNO                                                      3
OBJECTID                                                      3
DESCRIP                                           Jaw and teeth
OLDNO                                                       158
OTHERNO                                                     NaN
PROVENANCE    Locality: Bluehill Falls, Me. Section number -...
SITENO                                                   42.001
SITE                                            Nevin Shellheap
ORIGIN                 United States | ME | Hancock | Blue Hill
NOTES         Douglas Byers (Principal Investigator, 00/00/3...
Name: 1013, dtype: object

In [21]:
n = len([n for n in os.listdir('./peabody_files/Accession Files/') if n[0]=='3'])
m = len(df[df['ACCESSNO']==3])

print('There are %d cards with Acc. No. 3, and %d rows with Acc. No. 3.' % (n,m))

There are 526 cards with Acc. No. 3, and 529 rows with Acc. No. 3.


For some reason, this row at index **1013** does not mention Cat. No. in `OBJECTID`. The `ACCESSNO` of this row is 3, but there are **526** possible cards whose `ACCESSNO` is 3. Thus, **it is unfeasible to determine which card this row corresponds to.** 

In [22]:
df = df.drop(1013) #Bye!

There are three patterns for the `OBJECTID` column:
- ACCNO/CATNO
- MONTH-CATNO
- CATNO-MONTH

We want to convert all of these into *just* CATNO. 

In [23]:
def get_catno(string):
    if '/' in string:
        return int(re.search('(?<=\/)\d+',string).group(0))
    return int(re.search('\d+', string).group(0))

df['CATEGORYNO'] = df['OBJECTID'].apply(get_catno)
df.sample(5)

ACCESSNO OBJECTID                                            DESCRIP  \
1027         4   4/1023           Large crudely worked piece of quartzite.   
930          2    2/412                            Piece of worked antler.   
111          3    3/592                                        Pot sherds.   
820          2    2/255                              1 chipped arrowpoint.   
264          3    3/758  Distal end of solid seal bone perferator with ...   

     OLDNO OTHERNO                                         PROVENANCE  \
1027     f     NaN  Locality - North shore of Upper Chestnut Hill ...   
930    NaN   SA/74  East end of Squibnocket Pond, Martha's Vineyar...   
111     60     NaN  Section number - 4E Trench Number - ID.4 In sh...   
820    NaN   S/151  East end of Squibnocket Pond, Martha's Vineyar...   
264    190     NaN  Section number - 7E Trench Number - IC.3 Excav...   

         SITENO                   SITE  \
1027  Site 1255          E.D Prey c7/1   
930       M49/2  Hornblower Shell Heap   
111      42.001        Nevin Shellheap   
820       M49/2  Hornblower Shell Heap   
264      42.001        Nevin Shellheap   

                                        ORIGIN  \
1027                                       NaN   
930                                        NaN   
111   United States | ME | Hancock | Blue Hill   
820                                        NaN   
264   United States | ME | Hancock | Blue Hill   

                                                  NOTES  CATEGORYNO  
1027                                                NaN        1023  
930                                                 NaN         412  
111                    July 2013 PastPerfect Conversion         592  
820                                                 NaN         255  
264   Douglas Byers (Principal Investigator, 00/00/3...         758

Looks good. To be very meticulous, I also want to check another assumption: that ACCNO from `OBJECTID` match with `ACCESSNO` column. 

In [24]:
def get_accno(string):
    if '/' in string:
        return int(re.match('\d+',string).group(0))
    month = re.search('[^-\d]+', string).group(0)
    return int(months_abbv.index(month)+1)

df['ACCESSNO_OBJ'] = df['OBJECTID'].apply(get_accno)
df.query('ACCESSNO_OBJ != ACCESSNO')

Empty DataFrame
Columns: [ACCESSNO, OBJECTID, DESCRIP, OLDNO, OTHERNO, PROVENANCE, SITENO, SITE, ORIGIN, NOTES, CATEGORYNO, ACCESSNO_OBJ]
Index: []

Another assumption is that all of the rows are unique to a ACCNO-CATNO pair.

In [25]:
df.duplicated(subset=['ACCESSNO','CATEGORYNO']).any()

False

Another assumption is that all of the rows have a corresponding image. Is this true?

In [26]:
def image_exists(x):
    return os.path.exists(card_path(x['ACCESSNO'], x['CATEGORYNO']))

t = df.apply(image_exists, axis=1)
len(t[t==False])

23

**23 rows** do not have an image of a card that corresponds to their data. These are useless to our purpose of training an OCR, so we drop those too.

In [27]:
i = t[t==False].index
df.drop(i, inplace=True)

Now, it is 100% definite that all of the rows in the `df` dataset have the correct values in the `ACCESSNO` and `CAtEGORYNO` columns, and that all the rows are unique to one card that exists.

### Reevaluating Which Other Fields to Pay Attention To
In the section above, we dropped a number of rows that did not fulfill the criteria: 
-  Could not be identified by Category and Accession number
- Did not correspond to an image. 

Because of this, I want to re-evaluate which columns I want to pay attention to.

In [28]:
df.isnull().sum()

ACCESSNO          0
OBJECTID          0
DESCRIP           0
OLDNO           519
OTHERNO         562
PROVENANCE        0
SITENO            1
SITE              1
ORIGIN          389
NOTES           546
CATEGORYNO        0
ACCESSNO_OBJ      0
dtype: int64

In [29]:
for col in df.columns:
    print(col, len(df[col].unique()), sep=' | ')

ACCESSNO | 7
OBJECTID | 1070
DESCRIP | 861
OLDNO | 517
OTHERNO | 362
PROVENANCE | 749
SITENO | 8
SITE | 10
ORIGIN | 6
NOTES | 35
CATEGORYNO | 1070
ACCESSNO_OBJ | 7


The columns **SITE** and **SITENO** remain interesting, due to its few variety and **one** NaN. I will start there.

### SITE and SITENO

In [30]:
pd.set_option('max_colwidth',200)
df[['ACCESSNO','CATEGORYNO','SITE', 'SITENO']].sample(10)

ACCESSNO  CATEGORYNO                         SITE     SITENO
609         2         104        Hornblower Shell Heap      M49/2
978         2         436        Hornblower Shell Heap      M49/2
291         3         790              Nevin Shellheap     42.001
820         2         255        Hornblower Shell Heap      M49/2
673         2         155        Hornblower Shell Heap      M49/2
123         3         606              Nevin Shellheap     42.001
146         3         629              Nevin Shellheap     42.001
570         1          49  Squibnocket Cliff Shellheap  Site 0213
13          3        1014              Nevin Shellheap     42.001
289         3         788              Nevin Shellheap     42.001

Questions:
- Where is the NaN? (1 NaN in SITE column, 1 NaN in SITENO column)
- What are the unique values of SITE? What are the unique values of SITENO?
- Do SITE and SITENO always have the same pair?
- Do the cards correspond exactly to text on SITE or SITENO?

#### NaN

In [31]:
# Display the row with NaN in SITE column.
df[df['SITE'].isnull()]

ACCESSNO OBJECTID  \
334         3    3/857   

                                                                                DESCRIP  \
334  Bit end of humped-back gouge. Original Accession Card: Battered hump-backed gouge.   

    OLDNO OTHERNO  \
334   462     NaN   

                                                                                                                                                   PROVENANCE  \
334  Section number - 8E Trench Number - IC.1 Excav Date - 00/00/36 Locality - Bluehill Falls, Me. Situation - Just under shells in mixed shells and pebbles.   

    SITENO SITE ORIGIN  \
334    NaN  NaN    NaN   

                                                                                 NOTES  \
334  Douglas Byers (Principal Investigator, 00/00/36) July 2013 PastPerfect Conversion   

     CATEGORYNO  ACCESSNO_OBJ  
334         857             3

As `SITENO` is NaN too, this is the only row in `df` with NaN for Site and Site. No. Why did this happen? Was the information not displayed on the card?

In [32]:
disp_card(3, 857)

The Work Duty student was probably exhausted that way. I will manually fill in this Missing Value, but only because there is only one, and does not take too much time at all.

In [33]:
df.set_value(334, 'SITE', 'Nevin Shell Heap.')
df.set_value(334, 'SITENO', 'Me.152/1')

ACCESSNO OBJECTID  \
0            3   3/1000   
1            3   3/1001   
2            3   3/1002   
3            3   3/1003   
4            3   3/1005   
5            3   3/1006   
6            3   3/1007   
7            3   3/1008   
8            3   3/1009   
9            3   3/1010   
10           3   3/1011   
11           3   3/1012   
12           3   3/1013   
13           3   3/1014   
14           3   3/1015   
15           3   3/1016   
16           3   3/1017   
18           3    3/491   
19           3    3/492   
20           3    3/493   
21           3    3/494   
22           3    3/495   
23           3    3/496   
24           3    3/497   
25           3    3/500   
26           3    3/501   
27           3    3/502   
28           3    3/503   
29           3    3/504   
30           3    3/505   
...        ...      ...   
1048         4   4/1045   
1049         4   4/1046   
1050         4   4/1047   
1051         4   4/1048   
1052         4   4/1049   
1053         4   4/1050   
1054         4   4/1051   
1055         4   4/1052   
1056         4   4/1053   
1057         4   4/1054   
1058         4   4/1055   
1059         4   4/1056   
1060         4   4/1057   
1061         4   4/1058   
1062         4   4/1059   
1063         4   4/1060   
1064         4   4/1061   
1065         5   5/1062   
1066         5   5/1063   
1067         5   5/1064   
1068         6   6/1065   
1069         6   6/1067   
1070         6   6/1068   
1071         6   6/1069   
1072         6   6/1070   
1073         7   7/1071   
1074         7   7/1072   
1075         7   7/1073   
1076         7   7/1074   
1086         6   6/1066   

                                                                                                                                     DESCRIP  \
0                                                                                                                                Pot sherds.   
1                                                                                                            Fragment of chipped arrowpoint.   
2                                                                                                                                Pot sherds.   
3                                                                                                                    Beaver tooth implement.   
4                                                            Barbed harpoon fragment. Appears to be unilateral. Neither tip nor butt extant.   
5                                                                                                             2 fragments stone arrow points   
6                                                                                                                      1 chipped stone knife   
7                                                                                                                      Butt of adze? Surface   
8                                                                                                               Edge of polished slate tool.   
9                                                                                                                             Large plummet.   
10                                                                                                      Point of large broken chipped knife.   
11                                                                                                                     Felsite (?) core. Sod   
12                                                         Small plummet, sandstone. In mixed shell and humus near edge of bank above beach.   
13                                                                                                                  Point of bone pike head.   
14                                                                                                          Fragment of worked beaver tooth.   
15                         

### Unique Values

In [34]:
print('Site', ', '.join(df['SITE'].unique()), sep=' : ')
print('Site No.', ', '.join(df['SITENO'].unique()), sep=' : ')

Site : Nevin Shellheap, Nevin Shell Heap., Squibnocket Cliff Shellheap, Hornblower Shell Heap, E.D Prey c7/1, Pomfret Landing, Pomfret landing, Mooseup Camp Site, Mooseup camp Site, Shellheap #1, Plum Island
Site No. : 42.001, Me.152/1, Site 0213, M49/2, Site 1255, Site 0820, Site 0821, Site 0843


The values that I inserted before for `df.loc[334]` show up as unique values. This is not good, because I added another category altogether; I'll replace with those that were existing before. For `SITENO`, it seems from glancing through the table that Nevin Shellheap rows also have 42.001 as their `SITENO`, so I'll use that.

In addition, there are two unique values for `Pompfret landing` and `Pompfret Landing`, in addition to `Mooseup Camp Site` and `Mooseup camp Site`, so I will fix this, too.

In [35]:
# Fixing my manually inputed values for row with NaN
df.set_value(334, 'SITE', 'Nevin Shellheap')
df.set_value(334, 'SITENO', '42.001')

# Eliminating duplicate site names with different capitalizations
df.replace('Pomfret landing', 'Pomfret Landing', inplace=True)
df.replace('Mooseup camp Site', 'Mooseup Camp Site', inplace=True)

# Checking
print('Site', ', '.join(df['SITE'].unique()), sep=' : ')
print('Site No.', ', '.join(df['SITENO'].unique()), sep=' : ')

Site : Nevin Shellheap, Squibnocket Cliff Shellheap, Hornblower Shell Heap, E.D Prey c7/1, Pomfret Landing, Mooseup Camp Site, Shellheap #1, Plum Island
Site No. : 42.001, Site 0213, M49/2, Site 1255, Site 0820, Site 0821, Site 0843


### SITE and SITENO correlation

With only unique values, there are as many unique values of `SITE` as `SITENO`, strengthening the theory that every `SITE` has a `SITENO` corresponding to it. 

In [36]:
print(len(df['SITE'].unique()), len(df['SITENO'].unique()))

7 7


The theory would be verified if there are as many combinations of `SITE` and `SITENO` as the unique values of `SITE` and `SITENO`. In short, if there are **7** combinations of SITE and SITENO.

In [37]:
def g(x):
    return x['SITE'] + x['SITENO']

t = df.apply(g, axis=1)
len(t.unique())

7

Now, it's confirmed that **every site has a unique site number.** 

### Matching text typed on cards
Earlier on I noticed that, while the cards indicated Nevin Shell Heap, the data showed Nevin Shellheap. In addition, when I typed in data manually for the row with NaN, I noticed that the `SITENO` column in the data did not correspond to card's text. As the objective of this Jupyter Notebook is to try to have a dataset matching the card's text **as closely as possible**, I need to change the styling of the dataset to resemble the text.

In [38]:
def disp_cards_at_indexes(*args):
    '''
    Pass in labels of index axes.
    '''
    k = []
    for i in args:
        acc, cat = df.loc[i,'ACCESSNO'], df.loc[i, 'CATEGORYNO']
        k.append((acc, cat))
        
    return disp_cards(*k)

def disp_cards_at_site(site, n=6):
    x = df[df['SITE'] == site]
    if n > len(x):
        n = len(x)
    i = x.sample(n).index.tolist()
    return disp_cards_at_indexes(*i)

#### Nevin Shell Heap.

In [39]:
sites = df['SITE'].unique()
snos = df['SITENO'].unique()
disp_cards_at_site(sites[0])

In [40]:
df.replace(sites[0], 'Nevin Shell Heap.', inplace=True)
df.replace(snos[0], 'Me.152/1', inplace=True)

#### Squibnocket Cliff.

In [41]:
disp_cards_at_site(sites[1])

In [42]:
df.replace(sites[1], 'Squibnocket Cliff.', inplace=True)
df.replace(snos[1], '', inplace=True)

#### Hornblower Shell Heap.

In [43]:
disp_cards_at_site(sites[2])

In [44]:
df.replace(sites[2], 'Hornblower Shell Heap.', inplace=True)
df.replace(snos[2], '', inplace=True)

#### E.D. Prey.

In [45]:
disp_cards_at_site(sites[3])

In [46]:
df.replace(sites[3], 'E.D. Prey.', inplace=True)
df.replace(snos[3], 'C 7/1', inplace=True)

#### Pomfret Landing.

In [47]:
disp_cards_at_site(sites[4])

In [48]:
df.replace(sites[4], 'Pomfret Landing.', inplace=True)
df.replace(snos[4], 'C 7/3', inplace=True)

#### Moosup Camp Site.

In [49]:
disp_cards_at_site(sites[5])

In [50]:
df.replace(snos[5], 'C 14/1', inplace=True)

There is a problem with Moosup Camp Site: some cards have a capitalized s, while others have a capitalized S. If there are not many cards with Mooseup Camp Site, then I don't mind manually changing them. 

In [51]:
len(df[df['SITE']==sites[5]])

6

There are only 6 cards, so the cards displayed above are all of the cards I need to see. Cards with Cat. No's 1069 and 1070 have lower 's', and the rest have capitalized 'S's. According to the table below, those rows have indexes 1071 and 1072.

In [52]:
df[df['SITE']==sites[5]]

ACCESSNO OBJECTID                       DESCRIP OLDNO OTHERNO  \
1068         6   6/1065               Quartzite chip.   NaN     NaN   
1069         6   6/1067            Base of arrowhead.   NaN     NaN   
1070         6   6/1068             Quartz spearhead.   NaN     NaN   
1071         6   6/1069  Triangular quartz arrowhead.   NaN     NaN   
1072         6   6/1070             Coiled pot sherd.   NaN     NaN   
1086         6   6/1066       Base of quartzite tool.   NaN     NaN   

                                                                                           PROVENANCE  \
1068  Situation - Unknown. Locality - Plainfiled township, Conn. 5/8 mi. N.E. of head of Moosup Pond.   
1069      Sitaution:Unknown. Locality: Plainfield township, Conn. 5/8 mi. N.E.of head of Moosup Pond.   
1070    Situation: Unknown. Locality: Plainfield township, Conn. 5/8 mi. N.E. of head of Moosup Pond.   
1071    Situation: Unknown. Locality: Plainfield township, Conn. 5/8 mi. N.E. of head of Moosup Pond.   
1072    Situation: Unknown. Locality: Plainfield township, Conn. 5/8 mi. N.E. of head of Moosup Pond.   
1086  Situation - Unknown. Locality - Plainfiled township, Conn. 5/8 mi. N.E. of head of Moosup Pond.   

      SITENO               SITE ORIGIN NOTES  CATEGORYNO  ACCESSNO_OBJ  
1068  C 14/1  Mooseup Camp Site    NaN   NaN        1065             6  
1069  C 14/1  Mooseup Camp Site    NaN   NaN        1067             6  
1070  C 14/1  Mooseup Camp Site    NaN   NaN        1068             6  
1071  C 14/1  Mooseup Camp Site    NaN   NaN        1069             6  
1072  C 14/1  Mooseup Camp Site    NaN   NaN        1070             6  
1086  C 14/1  Mooseup Camp Site    NaN   NaN        1066             6

In [53]:
df.replace(sites[5], 'Moosup Camp Site.', inplace=True)
for i in [1071, 1072]:
    df.set_value(i, 'SITE', 'Moosup Camp site.')

#### Shell Heap \#1.

In [54]:
disp_cards_at_site(sites[6])

In [55]:
df.replace(sites[6], 'Shell Heap #1.', inplace=True)
df.replace(snos[6], '', inplace=True)

# Wrap Up
The last thing I'll do is to change the names of the indexes into the PDF filenames. Finally, I'll save the final processed Dataframe into a CSV. 

In [61]:
df.set_index(df.apply(lambda x: '%d_%04d'%(x['ACCESSNO'], x['CATEGORYNO']),axis=1), inplace=True)
final_df = df[['ACCESSNO','CATEGORYNO','SITE','SITENO']]
final_df.sample(6)

ACCESSNO  CATEGORYNO                    SITE    SITENO
1_0041         1          41      Squibnocket Cliff.          
3_0626         3         626       Nevin Shell Heap.  Me.152/1
3_0695         3         695       Nevin Shell Heap.  Me.152/1
3_0992         3         992       Nevin Shell Heap.  Me.152/1
4_1049         4        1049              E.D. Prey.     C 7/1
2_0393         2         393  Hornblower Shell Heap.

In [ ]:
final_df.to_csv('transcription.csv')

### Bonus: Is there a set Locality to every Site?
It turned out that the CSV data would actually be useless in training the OCR, but I did start to explore whether I can account for 'Locality' fields on the cards as well.

In [64]:
t = df[df.SITE=='Hornblower Shell Heap.'].apply(lambda x: "East end of Squibnocket Pond, Martha's Vineyard, Mass." in x.PROVENANCE, axis=1)
t[t==False]

Series([], dtype: bool)

In [65]:
t = df[df.SITE=='Nevin Shell Heap.'].apply(lambda x: "Blue" in x.PROVENANCE or "Blue" in x.ORIGIN, axis=1)
t[t==False].index

Index([], dtype='object')

From above I can see that **all artifacts found in Hornblower Shell heap have the Locality of East end~Mass.** and artifacts found in **Nevil Shepp Heap have a Locality of Bluehill Falls, Me.**